In [1]:
import cv2 as cv
import numpy as np
import os

In [2]:
def show_image(title,image):
    image=cv.resize(image,(0,0),fx=0.15,fy=0.15)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
def extrage_careu(image):
    frame_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    lh, ls, lv = 25, 0, 0  
    uh, us, uv = 255, 255, 255  


    lower_bound = np.array([lh, ls, lv])
    upper_bound = np.array([uh, us, uv])

    mask = cv.inRange(frame_hsv, lower_bound, upper_bound)

    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    max_area = 0
    for i in range(len(contours)):
        if len(contours[i]) > 3:
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point
                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1]:
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis=1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]

            if cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width, height = 2400, 2400
    image_copy = image.copy()
    if max_area > 0:
        cv.circle(image_copy, tuple(top_left), 20, (0, 0, 255), -1)
        cv.circle(image_copy, tuple(top_right), 20, (0, 0, 255), -1)
        cv.circle(image_copy, tuple(bottom_left), 20, (0, 0, 255), -1)
        cv.circle(image_copy, tuple(bottom_right), 20, (0, 0, 255), -1)

    if max_area > 0:
        puzzle = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")
        destination_of_puzzle = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype="float32")
        M = cv.getPerspectiveTransform(puzzle, destination_of_puzzle)
        result = cv.warpPerspective(image, M, (width, height))

        x_start, x_end = 320, 2082  
        y_start, y_end = 320, 2082

        cropped_image = result[y_start:y_end, x_start:x_end]

        result2 = cv.resize(cropped_image, (2030,2030))
        return result2
    cv.waitKey(0)
    cv.destroyAllWindows()
    return None

In [4]:
lines_horizontal=[]
for i in range(0,2031,145):
    l=[]
    l.append((0,i))
    l.append((2029,i))
    lines_horizontal.append(l)

lines_vertical=[]
for i in range(0,2031,145):
    l=[]
    l.append((i,0))
    l.append((i,2029))
    lines_vertical.append(l)

In [5]:
def find_largest_region_center(mask):
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None

    largest_contour = max(contours, key=cv.contourArea)

    moments = cv.moments(largest_contour)

    if moments["m00"] != 0:
        center_x = int(moments["m10"] / moments["m00"])
        center_y = int(moments["m01"] / moments["m00"])
        return (center_x, center_y)
    else:
        return None


In [6]:
def find_grid_cell(center):
    cell_size = 2030 // 14
    center_x, center_y = center
    row = center_y // cell_size 
    col = center_x // cell_size  
    column_letter = chr(ord('A') + col) 
    return f"{row + 1}{column_letter}"

In [7]:
def process_images_and_find_center(img1, img2):
    img1 = cv.cvtColor(img1, cv.COLOR_BGR2HSV)
    img2 = cv.cvtColor(img2, cv.COLOR_BGR2HSV)

    h, s, v = cv.split(img1)
    h1, s1, v1 = cv.split(img2)

    diff = cv.absdiff(v, v1)
    diff_blur = cv.GaussianBlur(diff, (7, 7), 0)
    #show_image("Diferența", diff_blur)

    _, diff_thresh = cv.threshold(diff_blur, 98, 255, cv.THRESH_BINARY) 

    diff_thresh = cv.erode(diff_thresh, None, iterations=3)
    diff_thresh = cv.dilate(diff_thresh, None, iterations=3)
    #show_image ("Diferența", diff_thresh)

    center = find_largest_region_center(diff_thresh)

    if center:
        return(find_grid_cell(center))
    else:
        return None


In [8]:
def process_all_images_in_directory(directory, output_directory, img_aux):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)  
    list = []
    files = [f for f in os.listdir(directory) if f.endswith('.jpg')]  

    img0 = extrage_careu(cv.imread(img_aux))
    img1 = extrage_careu(cv.imread(os.path.join(directory, files[0])))
    x1 = process_images_and_find_center(img0, img1)
    patch = decupeaza_patrat(img1, x1)
    x2 = clasifica_cifra(patch)

    output_filename = os.path.splitext(files[0])[0] + ".txt"  
    with open(os.path.join(output_directory, output_filename), "w") as f:
        f.write(f"{x1} {x2}\n")
        list.append(f"{x1} {x2}")
    
    for i in range(len(files) - 1):
        if "01.jpg" in files[i + 1]:
            img1 = extrage_careu(cv.imread(img_aux))
        else: 
            img1 = extrage_careu(cv.imread(os.path.join(directory, files[i])))
        
        img2 = extrage_careu(cv.imread(os.path.join(directory, files[i + 1])))  

        x1 = process_images_and_find_center(img1, img2)
        patch = decupeaza_patrat(img2, x1)
        x2 = clasifica_cifra(patch)

        output_filename = os.path.splitext(files[i + 1])[0] + ".txt"
        with open(os.path.join(output_directory, output_filename), "w") as f:
            f.write(f"{x1} {x2}\n")
            list.append(f"{x1} {x2}")
    return list


In [9]:
def write_combined_txt_files(directory):
    list = []

    
    for file_name in os.listdir(directory):
        if file_name.endswith('.txt'):  
            file_path = os.path.join(directory, file_name)
            with open(file_path, 'r', encoding='utf-8') as infile:  
                content = infile.read().strip()  
            
                list.append(content)

    result = [item for item in list if not item.startswith('Pl')]
             
    return result


In [10]:
def compare_lists_percentage(list1, list2):

    matching_elements = sum(1 for a, b in zip(list1, list2) if a == b)
    
    match_percentage = (matching_elements / max(len(list1), len(list2))) * 100
    
    return match_percentage

In [11]:
# list1 = write_combined_txt_files('data/antrenare', 'combined_output.txt')
# list2 = process_all_images_in_directory('data/antrenare')
# print(len(list1))
# print(len(list2))
# print(compare_lists_percentage(list1, list2))

In [12]:
def determina_numar_cifre(patch):

    if len(patch.shape) == 3:
        patch = cv.cvtColor(patch, cv.COLOR_BGR2GRAY)
    patch_bin = cv.adaptiveThreshold(
        patch, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 11, 2
    )
    
    contours, _ = cv.findContours(patch_bin, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
   
    height, width = patch_bin.shape
    contururi_valide = []
    for contour in contours:
        if cv.contourArea(contour) > 700:  
            x, y, w, h = cv.boundingRect(contour)
            if x > 0 and y > 0 and x + w < width and y + h < height:
                contururi_valide.append(contour)

    if len(contururi_valide) >= 2:
        x = 2
    else:
        x = 1
    
    return x

In [13]:
def clasifica_cifra(patch):
    maxi = -np.inf
    poz = -1

    x = determina_numar_cifre(patch)
    patch = cv.cvtColor(patch, cv.COLOR_BGR2GRAY)
    _, patch = cv.threshold(patch, 150, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
 
    patch = cv.erode(patch, None, iterations=2)
    patch = cv.dilate(patch, None, iterations=2)

    template_single_digit = [f"{i}.jpg" for i in range(10)] 

    template_double_digit = [filename for filename in os.listdir('templates') if len(filename.split('.')[0]) == 2]

    if x == 1:
        template_list = template_single_digit
    elif x == 2:
        template_list = template_double_digit

    for template_name in template_list:
        template_path = os.path.join('templates', template_name)

        if os.path.exists(template_path):
            img_template = cv.imread(template_path)

            img_template = cv.cvtColor(img_template, cv.COLOR_BGR2GRAY)

            _, img_template = cv.threshold(img_template, 150, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
         
            if patch[0].shape >= img_template.shape and patch[1].shape >= img_template.shape:
                corr = cv.matchTemplate(img_template,patch, cv.TM_CCOEFF_NORMED)
                corr = np.max(corr)

                if corr > maxi:
                    maxi = corr
                    poz = int(template_name.split('.')[0]) 
            
            else:
                print(f"Imaginea {template_name} are dimensiuni diferite față de patch.")
            

    return poz


In [14]:
def decupeaza_patrat(img, pozitie):
   
    dimensiune=145

    linie = int(pozitie[:-1]) - 1 
    coloana = ord(pozitie[-1]) - ord('A')  

    x_start = coloana * dimensiune
    x_end = x_start + dimensiune
    y_start = linie * dimensiune
    y_end = y_start + dimensiune

    decupaj = img[y_start:y_end, x_start:x_end]

    return decupaj


In [15]:


def extrage_patrate(img, folder_destinatie, dimensiune_matrice=14):
  
    img = extrage_careu(img)
   
    height, width, _ = img.shape

    patrat_height = height // dimensiune_matrice
    patrat_width = width // dimensiune_matrice

    if not os.path.exists(folder_destinatie):
        os.makedirs(folder_destinatie)

    for linie in range(1, dimensiune_matrice + 1):  
        for coloana_index in range(dimensiune_matrice):  
            x_start = coloana_index * patrat_width
            x_end = (coloana_index + 1) * patrat_width
            y_start = (linie - 1) * patrat_height
            y_end = linie * patrat_height
            
            patrat = img[y_start:y_end, x_start:x_end]

            nume_patrat = f"{linie}{chr(65 + coloana_index)}.jpg"  
            
            calea_patrat = os.path.join(folder_destinatie, nume_patrat)
            
            cv.imwrite(calea_patrat, patrat)
            print(f"Salvat: {calea_patrat}")




In [16]:
def proceseaza_imagini_in_folder(input_folder):
    
    output_folder = 'templatesnoi'

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    filenames = sorted(os.listdir(input_folder))
    for idx, filename in enumerate(filenames):
        if filename.endswith(('.jpg', '.png')):
            img_path = os.path.join(input_folder, filename)
            img = cv.imread(img_path)
            
            if img is None:
                print(f"Imaginea {filename} nu a putut fi încărcată.")
                continue
            
            output_path = os.path.join(output_folder, f"{idx-14}.jpg")
            rez = decupeaza_numar(img)
            cv.imwrite(output_path, rez)
            print(f"Imaginea {filename} a fost procesată și salvată ca {idx}.jpg.")

In [17]:
input_folder = 'testare'
output_folder = 'rezultatele_mele1'
img_aux = '01.jpg'

process_all_images_in_directory(input_folder,output_folder,img_aux)

['9G 2',
 '9H 6',
 '8I 7',
 '7I 1',
 '9I 8',
 '10I 15',
 '9J 14',
 '8J 11',
 '7J 2',
 '6J 13',
 '10J 25',
 '10K 40',
 '11I 7',
 '10H 24',
 '10G 5',
 '11H 30',
 '11G 10',
 '11F 20',
 '9F 12',
 '11E 2',
 '8F 7',
 '7F 19',
 '11D 18',
 '11C 9',
 '11B 9',
 '11A 81',
 '6G 4',
 '12H 6',
 '13H 36',
 '14H 12',
 '12G 2',
 '12I 8',
 '13I 1',
 '13G 35',
 '14G 70',
 '14I 28',
 '12F 4',
 '10F 16',
 '9E 10',
 '10E 21',
 '12E 2',
 '13E 1',
 '14E 3',
 '13F 5',
 '13D 6',
 '14F 1',
 '14D 4',
 '12D 2',
 '14C 7',
 '13C 7',
 '7F 2',
 '7E 3',
 '7D 6',
 '8F 7',
 '9F 9',
 '8I 7',
 '7I 3',
 '8J 11',
 '10F 16',
 '11F 25',
 '12F 9',
 '9H 6',
 '9G 4',
 '9I 10',
 '7J 6',
 '9J 17',
 '10J 28',
 '9K 27',
 '7K 9',
 '7L 54',
 '7M 6',
 '7N 9',
 '6J 5',
 '9L 10',
 '5J 1',
 '6H 6',
 '6G 4',
 '6I 10',
 '5I 13',
 '6F 14',
 '5H 8',
 '4H 48',
 '6K 15',
 '6L 10',
 '6M 5',
 '5K 24',
 '4K 9',
 '6E 10',
 '8E 21',
 '6N 2',
 '8N 7',
 '6D 4',
 '8D 10',
 '7C 2',
 '6C 6',
 '8C 4',
 '9C 2',
 '10C 8',
 '9N 2',
 '6B 10',
 '7I 3',
 '8I 7',

In [18]:
# list1 = write_combined_txt_files('data/evaluare/fisiere_solutie/331_Alexe_Bogdan')
# list2 = process_all_images_in_directory('data/evaluare/fake_test','data/evaluare/rezultatele_mele1','data/imagini_auxiliare/01.jpg')
# print(len(list1))
# print(len(list2))
# print(compare_lists_percentage(list1, list2))

In [19]:
# print(list1)
# print(list2)
# mismatched_positions = []
# for i in range(len(list1)):
#     if list1[i] != list2[i]:
#         mismatched_positions.append(i)
# print(mismatched_positions)